# Notebook to accompany the forecasting module MATH6011M at the University of Southampton.
https://www.southampton.ac.uk/courses/modules/math6011

aeon is a python toolkit for time series machine learning, including forecasting. We
will show you how to use aeon to enhance your understanding of the module. These
examples are based on the material provided for the module https://link.springer.com/article/10.1007/s43069-022-00179-z and have been developed in collaboration with the module organiser, Alain Zemkoho.

If you have any questions, please join our slack and ask in the #math6011 channel, we
 are more than happy to help.


## 1. Getting started with aeon

Install, environment and import the necessary packages.




## Plotting and basic analysis (Section 2.2)

Load data, correlation, etc


## Basic Usage, forecasting horizon, naive forecasting

Naive forecasting, MAPE, etc

 ## Performance Metrics (section 3.1)

## Exponential Smoothing (section 3.2)

## ARIMA Models (section 4.1)


## Model selection (section 4.2)

## Regression in forecasting (section 5)

## Case study:

Leo?


